# Notebook to loop over events generated in Pythia's c++ version


In [11]:
import numpy as np
import sys
import dimod
import matplotlib.pyplot as plt
import os
import re
import neal

from dwave.system import EmbeddingComposite, DWaveSampler, ReverseAdvanceComposite
from dwave_qbsolv import QBSolv
from pyqubo import Spin, Array
from embedding_generator import generate_embedding

In [16]:
# Define variable here

# Observed big difference in obtained minimum when increasing the number of reads from 100 to 1000
num_reads = 1000
num_events = 1000
path = '/home/andrea/pythiaEvents/'

theta = np.pi/2.

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
sa_sampler = neal.SimulatedAnnealingSampler()

In [30]:

with open('/home/andrea/pythiaEvents/DWaveResults_13_934.dat', 'w') as fout:
    for ev in range(934, num_events):
        print(ev)

        fname = path + 'Event_' + str(ev) + '.dat'
        f = open(fname)

        pat = re.compile(r"\s+")

        g = list(zip(*[ pat.sub(" ",x.strip()).split() for x in f]))

        px = np.array([ float(x) for x in g[0]])
        py = np.array([ float(x) for x in g[1]])
        pz = np.array([ float(x) for x in g[2]])
        e = np.array([ float(x) for x in g[3]])

        # Generate qubo

        n_part = len(px)
        #print(n_part)
        s = Array.create('s', shape=n_part, vartype='BINARY')

        # Create an array of zeroes for qubo matrix coefficients
        coeff = [[0] * n_part for _ in range(n_part)] 

        for i in range(0,n_part):
            for j in range(0,n_part):
                coeff[i][j] = (px[i]*px[j] +py[i]*py[j] + pz[i]*pz[j] - e[i]*e[j] * np.cos(theta))/(1 - np.cos(theta))

        # Construct Hamiltonian
        H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

        # Compile model using pyqubo

        model = H.compile()
        qubo, offset = model.to_qubo()

        # Submit problem to D-Wave
        #response = QBSolv().sample_qubo(qubo)#
        response = sampler.sample_qubo(qubo,num_reads=100)
        sampler_reverse = ReverseAdvanceComposite(sampler)
        schedule = [[[0.0, 1.0], [t, 0.5], [20, 1.0]] for t in (5, 10, 15)]
        sampleset = sampler_reverse.sample_qubo(qubo, anneal_schedules=schedule, num_reads=100,reinitialize_state=True) 


        fnameout = '/home/andrea/pythiaEvents/Event_%s_qsolutions.dat' %ev
        with open(fnameout, 'w') as f:
            for p in range(n_part):
                key = 's[%s]' %p
                f.write("%s\n" %(response.first.sample[key]))
    
    #print('Finished annealing run on Advantage 1.1')

        #print(response.first.energy)

        response_sa = sa_sampler.sample_qubo(qubo)

    #print('Finished simmulated annealing routine')

    #print(response_sa.first.energy)

        sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])
    
        #print(sum_mom)
        thrust = 2.*np.sqrt(-1.*response.first.energy)/sum_mom
        print(thrust)

        rev = 2.*np.sqrt(-1.*sampleset.first.energy)/sum_mom
        print(rev)

        thrust_sa = 2.*np.sqrt(-1.*response_sa.first.energy)/sum_mom
        fout.write("%s  %s  %s \n" %(thrust,thrust_sa,rev))
    
    #print(response.data_vectors['sample'][0])
    # Store solutions

    #qpu_solutions = []

    #for datum in response.data(['sample','energy']):
    #    print(datum.energy)
    #    qpu_solutions.append(datum.sample)


934
0.922397585837935
0.9134190879637378
935
0.8583937586486055
0.9309311697835483
936
0.8842106592415603
0.9137502729468436
937
0.9646876932662592
0.9710041186527221
938
0.9814090320811351
0.9783367140166932
939
0.8957459251739346
0.9602426676695881
940
0.9248276904987323
0.9477621209724091
941
0.9562319305978267
0.9678543693699804
942
0.9302761501333623
0.9672477867828372
943
0.9491496462708391
0.9492615050528026
944
0.9289878690180745
0.9577975000974716
945
0.910182968225047
0.931756226405164
946
0.8259796795758084
0.8597133348852598
947
0.9080984347780651
0.9321629648165309
948
0.937884566075561
0.9410199656684086
949
0.979156041605273
0.979156041605273
950
0.9052819373666617
0.9082525604448768
951
0.9558436533731134
0.9579808518403375
952
0.9563240728945536
0.948903326044757
953
0.9793162605919399
0.9783429485152125
954
0.7388922591352144
0.7393774156907923
955
0.9674276312456564
0.9691231867262473
956
0.9531147967309217
0.9608282067116005
957
0.9470550401130018
0.9530160613123203

In [10]:
with open('/home/andrea/pythiaEvents/Event_0_qsolutions.dat', 'w') as f:
    for p in range(n_part):
        key = 's[%s]' %p
        f.write("%s\n" %(response.first.sample[key]))

In [17]:
p = 0
response.first.sample['s[7]']

0

In [ ]:
thrust = sum([ -1.0*coeff[i][j]*response.first.sample['s['+str(i)+']']*response.first.sample['s['+str(j)+']'] for i in range(0,n_part) for j in range(0,n_part)])
print(thrust)

In [24]:
sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])

In [25]:
print(sum_mom)

90.86611912138152


In [ ]:
-thrust/(2*sum_mom)